In [1]:
import os

In [2]:
%pwd

'd:\\MLpipeline\\mlpipeline-example\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLpipeline\\mlpipeline-example'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [7]:
# import 

from src.mlProject.constants import *
# whatever things are inside constants import everything

from src.mlProject.utils.common import read_yaml,create_directories,save_json


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [9]:
import pandas as pd
import numpy as np
import os
from src.mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse



In [10]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, predicted):
        # Handle the 'YES'/'NO' labels in a binary classification context
        try:
            precision = precision_score(actual, predicted, average="binary", pos_label='YES')
            recall = recall_score(actual, predicted, average="binary", pos_label='YES')
            f1 = f1_score(actual, predicted, average="binary", pos_label='YES')
        except ValueError as e:
            print(f"Error calculating metrics: {e}")
            raise

        accuracy = accuracy_score(actual, predicted)
        confusion = confusion_matrix(actual, predicted)
        report = classification_report(actual, predicted, output_dict=True)  # Use output_dict for tabular format

        return accuracy, precision, recall, f1, confusion, report

    def save_results(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Split features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # Generate predictions
        predicted_labels = model.predict(test_x)

        # Calculate metrics
        accuracy, precision, recall, f1, confusion, report_dict = self.eval_metrics(test_y, predicted_labels)

        # Convert classification_report to DataFrame
        report_df = pd.DataFrame(report_dict).transpose()

        # Save the classification report as a CSV file
        report_csv_path = os.path.join(self.config.root_dir, "classification_report.csv")
        report_df.to_csv(report_csv_path, index=True)

        # Save other metrics
        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion_matrix": confusion.tolist()  # Convert to list for JSON serialization
        }

        # Format the JSON output with indentation
        formatted_scores = json.dumps(scores, indent=4)

        # Save the formatted metrics to a JSON file
        with open(self.config.metric_file_name, 'w') as json_file:
            json_file.write(formatted_scores)

        print(f"Model evaluation completed. Metrics saved to: {self.config.metric_file_name}")
        print(f"Classification report saved to: {report_csv_path}")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-12-23 15:50:37,929: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-23 15:50:37,933: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-23 15:50:37,938: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-23 15:50:37,941: INFO: common: created directory at: artifacts]
[2024-12-23 15:50:37,944: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-23 15:50:37,966: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
